In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

BASE_DIR = Path(r'U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie')
YEAR = 2009
IN_PATH = BASE_DIR / 'data' / 'processed' / f'merged_wind_cloud_{YEAR}_day_night.csv'
OUT_PATH = BASE_DIR / 'data' / 'processed' / f'merged_wind_cloud_{YEAR}_day_night_ausbreitung.csv'

TZ_NAME = 'UTC'
try:
    pd.Timestamp('2000-01-01', tz=TZ_NAME)
except Exception:
    TZ_NAME = 'UTC'


In [2]:
df = pd.read_csv(IN_PATH)
# Use the provided local timestamps from the merged dataset
df['timestamp_local'] = pd.to_datetime(df['timestamp_local'], errors='coerce')
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Add an extra local hour (2010-01-01 00:00) so the series still covers full 2009
last_row = df.iloc[-1].copy()
extra_row = last_row.copy()
ts_step = pd.Timedelta(hours=1)
ts_day = pd.Timedelta(days=1)

next_ts = pd.to_datetime(last_row['timestamp_local'], errors='coerce') + ts_step
local_last = pd.to_datetime(last_row['timestamp_local'], errors='coerce')
if getattr(local_last, 'tzinfo', None) is None:
    local_last = local_last.tz_localize(TZ_NAME, ambiguous='NaT', nonexistent='shift_forward')

extra_row['timestamp'] = next_ts.strftime('%Y-%m-%d %H:%M:%S')
extra_row['timestamp_local'] = (local_last + ts_step).isoformat()
sa_last = pd.to_datetime(last_row['SA'], errors='coerce')
su_last = pd.to_datetime(last_row['SU'], errors='coerce')
extra_row['SA'] = (sa_last + ts_day).isoformat() if pd.notna(sa_last) else None
extra_row['SU'] = (su_last + ts_day).isoformat() if pd.notna(su_last) else None
extra_row['MESS_DATUM'] = int(pd.to_datetime(next_ts, errors='coerce').strftime('%Y%m%d%H'))
df = pd.concat([df, pd.DataFrame([extra_row])], ignore_index=True)

def _ensure_tz(series, tz_name):
    """
    Coerce any input to tz-aware datetimes in the given zone.
    Handles Series or array-like and tolerates invalids (-> NaT).
    """
    ts = pd.to_datetime(series, errors='coerce')
    if isinstance(ts, pd.Series) and not is_datetime64_any_dtype(ts):
        # Mixed timezone offsets become object dtype; parse with utc to keep .dt access
        ts = pd.to_datetime(series, errors='coerce', utc=True)
    if isinstance(ts, pd.Series):
        tzinfo = getattr(ts.dt, 'tz', None)
        if tzinfo is None:
            return ts.dt.tz_localize(tz_name, ambiguous='NaT', nonexistent='shift_forward')
        return ts.dt.tz_convert(tz_name)
    if isinstance(ts, pd.DatetimeIndex):
        if ts.tz is None:
            return ts.tz_localize(tz_name, ambiguous='NaT', nonexistent='shift_forward')
        return ts.tz_convert(tz_name)
    ts_idx = pd.DatetimeIndex(ts)
    if ts_idx.tz is None:
        return ts_idx.tz_localize(tz_name, ambiguous='NaT', nonexistent='shift_forward')
    return ts_idx.tz_convert(tz_name)

# Respect provided local timestamps from the merged dataset
local_ts = _ensure_tz(df['timestamp_local'], TZ_NAME)
df['timestamp_local'] = local_ts
df['SA'] = _ensure_tz(df['SA'], TZ_NAME)
df['SU'] = _ensure_tz(df['SU'], TZ_NAME)

# Normalize day_night to booleans (True = day, False = night)
if df['day_night'].dtype != bool:
    df['day_night'] = (
        df['day_night']
        .astype(str)
        .str.strip()
        .str.lower()
        .map({'true': True, 'false': False, '1': True, '0': False})
    )


In [3]:
# Bin wind speed and cloud cover according to the table
invalid_wind = (df['wind_speed_ms'] == -999) | (df['wind_dir_deg'] == -999)
wind_speed_clean = df['wind_speed_ms'].where(~invalid_wind)

wind_bins = [-np.inf, 1.2, 2.3, 3.3, 4.3, np.inf]
wind_labels = ['<=1.2', '1.3-2.3', '2.4-3.3', '3.4-4.3', '>=4.4']
wind_bin = pd.cut(wind_speed_clean, bins=wind_bins, labels=wind_labels)

day_cloud_bin = pd.cut(
    df['cloud_cover_oktas'], bins=[-np.inf, 2, 5, 8], labels=['0-2', '3-5', '6-8']
)
night_cloud_bin = pd.cut(
    df['cloud_cover_oktas'], bins=[-np.inf, 6, 8], labels=['0-6', '7-8']
)

mapping_day = {
    ('<=1.2', '0-2'): 'IV',
    ('<=1.2', '3-5'): 'IV',
    ('<=1.2', '6-8'): 'IV',
    ('1.3-2.3', '0-2'): 'IV',
    ('1.3-2.3', '3-5'): 'IV',
    ('1.3-2.3', '6-8'): 'III/2',
    ('2.4-3.3', '0-2'): 'IV',
    ('2.4-3.3', '3-5'): 'IV',
    ('2.4-3.3', '6-8'): 'III/2',
    ('3.4-4.3', '0-2'): 'IV',
    ('3.4-4.3', '3-5'): 'III/2',
    ('3.4-4.3', '6-8'): 'III/2',
    ('>=4.4', '0-2'): 'III/2',
    ('>=4.4', '3-5'): 'III/1',
    ('>=4.4', '6-8'): 'III/1',
}

mapping_night = {
    ('<=1.2', '0-6'): 'I',
    ('<=1.2', '7-8'): 'II',
    ('1.3-2.3', '0-6'): 'I',
    ('1.3-2.3', '7-8'): 'II',
    ('2.4-3.3', '0-6'): 'II',
    ('2.4-3.3', '7-8'): 'III/1',
    ('3.4-4.3', '0-6'): 'III/1',
    ('3.4-4.3', '7-8'): 'III/1',
    ('>=4.4', '0-6'): 'III/1',
    ('>=4.4', '7-8'): 'III/1',
}

df['klasse_kt'] = [mapping_day.get(k) for k in zip(wind_bin.astype(object), day_cloud_bin.astype(object))]
df['klasse_kn'] = [mapping_night.get(k) for k in zip(wind_bin.astype(object), night_cloud_bin.astype(object))]
df['ausbreitungsklasse_base'] = np.where(df['day_night'] == True, df['klasse_kt'], df['klasse_kn'])

# Time windows around sunrise (SA) and sunset (SU)
ts_utc = df['timestamp_local']
sa_utc = df['SA']
su_utc = df['SU']
delta_sa = pd.TimedeltaIndex(ts_utc - sa_utc).total_seconds() / 3600
delta_su = pd.TimedeltaIndex(ts_utc - su_utc).total_seconds() / 3600

window = pd.Series(index=df.index, dtype='object')
window.loc[(delta_sa >= 0) & (delta_sa < 1)] = 'SA+1..SA+2'
window.loc[(delta_sa >= 1) & (delta_sa < 2)] = 'SA+1..SA+2'
window.loc[(delta_sa >= 2) & (delta_sa < 3)] = 'SA+2..SA+3'
window.loc[(delta_su >= -2) & (delta_su < -1)] = 'SU-2..SU-1'
window.loc[(delta_su >= -1) & (delta_su < 0)] = 'SU-1..SU'
window.loc[(delta_su >= 0) & (delta_su < 1)] = 'SU..SU+1'

transition_map = {
    ('I', 'IV', 'SA+1..SA+2'): ('I', 'II', 'a'),
    ('I', 'IV', 'SA+2..SA+3'): ('II', None, None),
    ('I', 'IV', 'SU-2..SU-1'): ('II', None, None),
    ('I', 'IV', 'SU-1..SU'): ('II', 'I', 'b'),
    ('I', 'IV', 'SU..SU+1'): ('I', 'II', 'a'),

    ('I', 'III/2', 'SA+1..SA+2'): ('II', None, None),
    ('I', 'III/2', 'SA+2..SA+3'): ('II', None, None),
    ('I', 'III/2', 'SU-2..SU-1'): ('III/1', None, None),
    ('I', 'III/2', 'SU-1..SU'): ('III/1', None, None),
    ('I', 'III/2', 'SU..SU+1'): ('I', 'II', 'a'),

    ('II', 'IV', 'SA+1..SA+2'): ('II', None, None),
    ('II', 'IV', 'SA+2..SA+3'): ('III/1', None, None),
    ('II', 'IV', 'SU-2..SU-1'): ('III/1', None, None),
    ('II', 'IV', 'SU-1..SU'): ('II', None, None),
    ('II', 'IV', 'SU..SU+1'): ('II', None, None),

    ('II', 'III/2', 'SA+1..SA+2'): ('III/1', None, None),
    ('II', 'III/2', 'SA+2..SA+3'): ('III/1', None, None),
    ('II', 'III/2', 'SU-2..SU-1'): ('III/1', None, None),
    ('II', 'III/2', 'SU-1..SU'): ('III/1', None, None),
    ('II', 'III/2', 'SU..SU+1'): ('II', None, None),

    ('III/1', 'IV', 'SA+1..SA+2'): ('III/1', None, None),
    ('III/1', 'IV', 'SA+2..SA+3'): ('III/2', None, None),
    ('III/1', 'IV', 'SU-2..SU-1'): ('III/2', None, None),
    ('III/1', 'IV', 'SU-1..SU'): ('III/1', None, None),
    ('III/1', 'IV', 'SU..SU+1'): ('III/1', None, None),

    ('III/1', 'III/2', 'SA+1..SA+2'): ('III/1', None, None),
    ('III/1', 'III/2', 'SA+2..SA+3'): ('III/1', None, None),
    ('III/1', 'III/2', 'SU-2..SU-1'): ('III/2', None, None),
    ('III/1', 'III/2', 'SU-1..SU'): ('III/2', None, None),
    ('III/1', 'III/2', 'SU..SU+1'): ('III/1', None, None),

    ('III/1', 'III/1', 'SA+1..SA+2'): ('III/1', None, None),
    ('III/1', 'III/1', 'SA+2..SA+3'): ('III/1', None, None),
    ('III/1', 'III/1', 'SU-2..SU-1'): ('III/1', None, None),
    ('III/1', 'III/1', 'SU-1..SU'): ('III/1', None, None),
    ('III/1', 'III/1', 'SU..SU+1'): ('III/1', None, None),
}


def _resolve_transition(kn, kt, win, base, month, wind, cloud):
    val = transition_map.get((kn, kt, win))
    if val is None:
        return base
    default, alt, rule = val
    if alt and rule == 'a':
        if (3 <= month <= 11) and (wind >= 1.3):
            return alt
    if alt and rule == 'b':
        if (month in (1, 2, 12)) and (wind < 1.3) and (cloud <= 6):
            return alt
    return default


months = pd.DatetimeIndex(df['timestamp_local']).month
wind_for_logic = wind_speed_clean

_df_wind = wind_for_logic
_df_cloud = df['cloud_cover_oktas']
_df_window = window
_df_kn = df['klasse_kn']
_df_kt = df['klasse_kt']
_df_base = df['ausbreitungsklasse_base']
df['ausbreitungsklasse'] = [
    _resolve_transition(kn, kt, win, base, m, w, c)
    for kn, kt, win, base, m, w, c in zip(
        _df_kn,
        _df_kt,
        _df_window,
        _df_base,
        months,
        _df_wind,
        _df_cloud,
    )
]

# Fallback for missing cloud_cover_oktas (use Table 4)
cloud_missing = df['cloud_cover_oktas'].isna()
wind_bins_nc = [-np.inf, 2.3, 3.3, np.inf]
wind_labels_nc = ['<=2.3', '2.4-3.3', '>=3.4']
wind_bin_nc = pd.cut(wind_for_logic, bins=wind_bins_nc, labels=wind_labels_nc)

window_nc = pd.Series(index=df.index, dtype='object')
window_nc.loc[(delta_sa < 1) | (delta_su >= 0)] = 'SU..SA+1'
window_nc.loc[(delta_sa >= 1) & (delta_sa < 3)] = 'SA+1..SA+3'
window_nc.loc[(delta_sa >= 3) & (delta_su <= -2)] = 'SA+3..SU-2'
window_nc.loc[(delta_su > -2) & (delta_su < 0)] = 'SU-2..SU'

mapping_nc = {
    ('<=2.3', 'SU..SA+1'): 'I',
    ('<=2.3', 'SA+1..SA+3'): 'II',
    ('<=2.3', 'SA+3..SU-2'): 'III/2',
    ('<=2.3', 'SU-2..SU'): 'III/1',
    ('2.4-3.3', 'SU..SA+1'): 'II',
    ('2.4-3.3', 'SA+1..SA+3'): 'III/1',
    ('2.4-3.3', 'SA+3..SU-2'): 'III/2',
    ('2.4-3.3', 'SU-2..SU'): 'III/1',
    ('>=3.4', 'SU..SA+1'): 'III/1',
    ('>=3.4', 'SA+1..SA+3'): 'III/1',
    ('>=3.4', 'SA+3..SU-2'): 'III/1',
    ('>=3.4', 'SU-2..SU'): 'III/1',
}

df['ausbreitungsklasse_no_cloud'] = [
    mapping_nc.get(k) for k in zip(wind_bin_nc.astype(object), window_nc)
]
df['ausbreitungsklasse'] = np.where(
    cloud_missing, df['ausbreitungsklasse_no_cloud'], df['ausbreitungsklasse']
)

# Summer uplift (VDI) rules
class_order = ['I', 'II', 'III/1', 'III/2', 'IV', 'V']
_next_class = {c: class_order[min(i + 1, len(class_order) - 1)] for i, c in enumerate(class_order)}

def _upgrade(series, mask):
    mapped = series.map(_next_class).fillna(series)
    return pd.Series(np.where(mask, mapped, series), index=series.index)

hours_local = pd.DatetimeIndex(df['timestamp_local']).hour
summer = months.isin([6, 7, 8])

mask_summer1 = summer & (hours_local >= 10) & (hours_local <= 16) & (
    (df['cloud_cover_oktas'] <= 6) |
    ((df['cloud_cover_oktas'] == 7) & (wind_for_logic < 2.4))
)
mask_summer2 = summer & (hours_local >= 12) & (hours_local <= 15) & (df['cloud_cover_oktas'] <= 5)
mask_may_sep = months.isin([5, 9]) & (hours_local >= 11) & (hours_local <= 15) & (df['cloud_cover_oktas'] <= 6)

df['ausbreitungsklasse'] = _upgrade(df['ausbreitungsklasse'], mask_summer1)
df['ausbreitungsklasse'] = _upgrade(df['ausbreitungsklasse'], mask_summer2)
df['ausbreitungsklasse'] = _upgrade(df['ausbreitungsklasse'], mask_may_sep)

# Winter rule after transitions: replace class IV with III/2 in Dec, Jan, Feb
winter_mask = months.isin([12, 1, 2])
df.loc[winter_mask & (df['ausbreitungsklasse'] == 'IV'), 'ausbreitungsklasse'] = 'III/2'

# Invalidate class when wind measurements are flagged as -999
null_class = invalid_wind
df.loc[null_class, 'ausbreitungsklasse'] = np.nan

missing = df['ausbreitungsklasse'].isna().sum()
print('missing class:', missing)
df[['wind_speed_ms', 'cloud_cover_oktas', 'day_night', 'ausbreitungsklasse_base', 'ausbreitungsklasse', 'ausbreitungsklasse_no_cloud']].head()


missing class: 0


,wind_speed_ms,cloud_cover_oktas,day_night,ausbreitungsklasse_base,ausbreitungsklasse,ausbreitungsklasse_no_cloud
0,1.5,5.0,False,I,I,I
1,1.6,NaN,False,None,I,I
2,2.0,NaN,False,None,I,I
3,1.7,NaN,False,None,I,I
4,1.9,NaN,False,None,I,I


In [4]:
df.to_csv(OUT_PATH, index=False)
print(OUT_PATH)

akterm_out = BASE_DIR / 'data' / 'processed' / f'akterm_{YEAR}_muenchen_stadt.akt'

# Build AKTERM-style export in UTC (no header row for columns)
ts_local = pd.to_datetime(df['timestamp_local'], errors='coerce')
ts_idx = pd.DatetimeIndex(ts_local)
if ts_idx.tz is None:
    ts_idx = ts_idx.tz_localize(TZ_NAME)
ts_utc = ts_idx.tz_convert('UTC')
ts_out = pd.DatetimeIndex(ts_utc).tz_localize(None)

# Keep only the target UTC year
mask_year = (ts_out >= pd.Timestamp(f'{YEAR}-01-01 00:00:00')) & (ts_out < pd.Timestamp(f'{YEAR + 1}-01-01 00:00:00'))
ts_out = ts_out[mask_year]
df_masked = df.loc[mask_year].reset_index(drop=True)

lines = []
kennzahl_map = {
    'I': '1',
    'II': '2',
    'III/1': '3',
    'III/2': '4',
    'IV': '5',
    'V': '6',
}
for ts, sid, wdir, ws, ak in zip(ts_out, df_masked['STATIONS_ID'], df_masked['wind_dir_deg'], df_masked['wind_speed_ms'], df_masked['ausbreitungsklasse']):
    sid_str = '' if pd.isna(sid) else f"{int(sid):05d}"
    wdir_str = '' if pd.isna(wdir) else f"{int(round(wdir))}"
    ws_str = '' if pd.isna(ws) else f"{int(round(ws * 10))}"
    ak_str = '' if pd.isna(ak) else ak
    if pd.isna(ak):
        main_ak = ''
        fehlwert = '-999'
    else:
        main_ak = kennzahl_map.get(str(ak), '')
        fehlwert = ''

    if pd.isna(ts):
        year = month = day = hour = minute = ''
    else:
        year = f"{ts.year:04d}"
        month = f"{ts.month:02d}"
        day = f"{ts.day:02d}"
        hour = f"{ts.hour:02d}"
        minute = f"{ts.minute:02d}"

    lines.append(
        f"AK {sid_str} {year} {month} {day} {hour} {minute} {wdir_str} {ws_str} {ak_str} {main_ak} {fehlwert}"
    )

header_lines = [
    '* AKTERM-Zeitreihe, Bayerisches Landesamt fuer Umwelt (LfU)',
    '* Station Muenchen-Stadt mit Bedeckung Muenchen-Stadt',
    f'* Zeitraum 01.01.{YEAR} - 31.12.{YEAR} (UTC).'
]
with akterm_out.open('w', encoding='utf-8') as f:
    for line in header_lines:
        f.write(line + '\n')
    for line in lines:
        f.write(line + '\n')

print(akterm_out)



U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie\data\processed\merged_wind_cloud_2009_day_night_ausbreitung.csv
U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie\data\processed\akterm_2009_muenchen_stadt.akt
